## Generate csv
As we only posses 2 experiments of the same type per user, it is necessary for us to somehow generate more datapoints. To accomplish that we segmented the data into n bins, lasting for m seconds, calculated basic statistical features (mean, sd, mcr) and save the results into separate csv files.

In [ ]:
from datetime import timedelta
import warnings

# Yes, yes, do not use this... we only have bunch of mean of empty slice and similar warnings
warnings.filterwarnings("ignore")

def generate_segmented_data_csv(seconds: int):
    """
    Generate csv file with calculated features, from data subsampled to the given time interval.
    """
    step = timedelta(seconds=seconds)
    seances = Seance.objects.filter(experiment__sequence_number=1, valid=True).order_by("start")
    
    print("Generating segmented data csv file with {} seconds intervals for {} seances.".format(seconds, seances.count()))
    
    file_name = "segmented_data_{}_seconds.csv".format(seconds)
    with open(file_name, "w") as csv_data:
        csv_data.write("user,seance,ax_me,ax_sd,ax_mcr,ax_mai,ay_me,ay_sd,ay_mcr,ay_mai,az_me,az_sd,az_mcr,az_mai,a_me,a_sd,a_mcr,a_mai,gx_me,gx_sd,gx_mcr,gy_me,gy_sd,gy_mcr,gz_me,gz_sd,gz_mcr,g_me,g_sd,g_mcr,fa_me,fa_sd,fa_mcr,fb_me,fb_sd,fb_mcr,fc_me,fc_sd,fc_mcr,fd_me,fd_sd,fd_mcr,ca_me,ca_sd,ca_min,ca_max,ca_mcr,cb_me,cb_sd,cb_min,cb_max,cb_mcr,cc_me,cc_sd,cc_min,cc_max,cc_mcr,cd_me,cd_sd,cd_min,cd_max,cd_mcr,c_me,c_sd,c_min,c_max,c_mcr,m_me,m_sd,m_min,m_max,m_jc,m_jr,m_jv,m_iji,ns_me,ns_sd,ns_sum,ns_jc,ns_jr,ns_jv,ns_iji,nr_me,nr_sd,nr_sum,nr_jc,nr_jr,nr_jv,nr_iji\n")
        count = 1
        for seance in seances[:]:
            print("-----------------------------------------------------------------------------")
            print("{} of {}".format(count, seances.count()))
            count += 1
            print(seance)
            start = seance.start
            end = seance.end
            data = list(load_data(seance.id, "accelerometer")) + list(load_data(seance.id, "gyroscope")) + list(load_data(seance.id, "force")) + list(load_data(seance.id, "cpu")) + [load_data(seance.id, "ram")] + list(load_data(seance.id, "net"))
            while start < end:
                sub_data = []
                for x in data:
                    try:
                        x[0].sensor
                        sub_data.append(x.filter(timestamp__range=(start, start + step)))
                    except IndexError:
                        sub_data.append([])

                # accelerometer
                ax_val, _, _, ax_me, ax_sd = process_signal(sub_data[0])
                ay_val, _, _, ay_me, ay_sd = process_signal(sub_data[1])
                az_val, _, _, az_me, az_sd = process_signal(sub_data[2])
                a_val, a_me, a_sd = join_accelerometer_signals(ax_val, ay_val, az_val)
                ax_mcr = mean_crossing_rate(ax_val, ax_me)
                ay_mcr = mean_crossing_rate(ay_val, ay_me)
                az_mcr = mean_crossing_rate(az_val, az_me)
                a_mcr = mean_crossing_rate(a_val, a_me)
                ax_mai = mean_acceleration_intensity(ax_val)
                ay_mai = mean_acceleration_intensity(ay_val)
                az_mai = mean_acceleration_intensity(az_val)
                a_mai = mean_acceleration_intensity(a_val)

                # gyroscope
                gx_val, _, _, gx_me, gx_sd = process_signal(sub_data[3])
                gy_val, _, _, gy_me, gy_sd = process_signal(sub_data[4])
                gz_val, _, _, gz_me, gz_sd = process_signal(sub_data[5])
                g_val, g_me, g_sd = join_accelerometer_signals(gx_val, gy_val, gz_val)
                gx_mcr = mean_crossing_rate(gx_val, gx_me)
                gy_mcr = mean_crossing_rate(gy_val, gy_me)
                gz_mcr = mean_crossing_rate(gz_val, gz_me)
                g_mcr = mean_crossing_rate(g_val, g_me)

                # force
                fa_val, _, _, fa_me, fa_sd = process_signal(sub_data[6])
                fb_val, _, _, fb_me, fb_sd = process_signal(sub_data[7])
                fc_val, _, _, fc_me, fc_sd = process_signal(sub_data[8])
                fd_val, _, _, fd_me, fd_sd = process_signal(sub_data[9])
                fa_mcr = mean_crossing_rate(fa_val, fa_me)
                fb_mcr = mean_crossing_rate(fb_val, fb_me)
                fc_mcr = mean_crossing_rate(fc_val, fc_me)
                fd_mcr = mean_crossing_rate(fd_val, fd_me)

                # cpu
                ca_val, _, _, ca_me, ca_sd = process_signal(sub_data[10])
                cb_val, _, _, cb_me, cb_sd = process_signal(sub_data[11])
                cc_val, _, _, cc_me, cc_sd = process_signal(sub_data[12])
                cd_val, _, _, cd_me, cd_sd = process_signal(sub_data[13])
                c_val, c_me, c_sd = join_cpu_signals(ca_val, cb_val, cc_val, cd_val)
                ca_min, ca_max, ca_mcr = get_cpu_stats(ca_val)
                cb_min, cb_max, cb_mcr = get_cpu_stats(cb_val)
                cc_min, cc_max, cc_mcr = get_cpu_stats(cc_val)
                cd_min, cd_max, cd_mcr = get_cpu_stats(cd_val)
                c_min, c_max, c_mcr = get_cpu_stats(c_val)

                # ram
                m_val, _, _, m_me, m_sd = process_signal(sub_data[14])
                derivatives, peaks = find_ram_jump(m_val)
                m_me, m_min, m_max, m_jc, m_jr, m_jv, m_iji = get_mem_stats(m_val, peaks, derivatives)

                # net
                ns_val, _, _, ns_me, ns_sd = process_signal(sub_data[15])
                nr_val, _, _, nr_me, nr_sd = process_signal(sub_data[16])
                ns_der, ns_pe = find_net_jump(ns_val)
                nr_der, nr_pe = find_net_jump(nr_val)
                ns_sum, ns_jc, ns_jr, ns_jv, ns_iji = get_net_stats(ns_val, ns_pe, ns_der)
                nr_sum, nr_jc, nr_jr, nr_jv, nr_iji = get_net_stats(nr_val, nr_pe, nr_der)
                
                write_row = ",".join([str(x) for x in [seance.user.id, seance.id, ax_me,ax_sd,ax_mcr,ax_mai,ay_me,ay_sd,ay_mcr,ay_mai,az_me,az_sd,az_mcr,az_mai,a_me,a_sd,a_mcr,a_mai, gx_me,gx_sd,gx_mcr,gy_me,gy_sd,gy_mcr,gz_me,gz_sd,gz_mcr,g_me,g_sd,g_mcr, fa_me,fa_sd,fa_mcr,fb_me,fb_sd,fb_mcr,fc_me,fc_sd,fc_mcr,fd_me,fd_sd,fd_mcr, ca_me,ca_sd,ca_min,ca_max,ca_mcr,cb_me,cb_sd,cb_min,cb_max,cb_mcr,cc_me,cc_sd,cc_min,cc_max,cc_mcr,cd_me,cd_sd,cd_min,cd_max,cd_mcr,c_me,c_sd,c_min,c_max,c_mcr, m_me,m_sd,m_min,m_max,m_jc,m_jr,m_jv,m_iji, ns_me,ns_sd,ns_sum,ns_jc,ns_jr,ns_jv,ns_iji,nr_me,nr_sd,nr_sum,nr_jc,nr_jr,nr_jv,nr_iji]])
                csv_data.write(write_row + "\n")
                start += step
 

# for seconds in [30, 15, 45, 60, 75, 90, 120, 10]:
for seconds in [150, 180]:
#     try:
    generate_segmented_data_csv(seconds=seconds)
#     except Exception as e:
#         print("ERROR")
#         print(e)
#         continue
  

## Apply LDA

In contrast to PCA which is an unsupervised method, LDA uses class labels to separate the data. As our data is labelled, this method should distinguish much better between different users. We will take a look at different time segment intervals, differnet number of users and some comparison to pca on the same data.

In [2]:
segment_intervals = [10, 15, 30, 45, 60, 75, 90, 120, 150, 180]
datas = []
for interval in segment_intervals:
    data_name = "segmented_data_{}_seconds.csv".format(interval)
    datas.append(read_csv(data_name).fillna(0))
warnings.filterwarnings("ignore")

### Evaluation of cluster quality

The result of used methods, either lda or pca is a point in n-dimensional space. Our goal is to maximize distance between clusters of point for different users, while retaining points of the same user close together. For that reason, we will use the Calinski-Harabasz Index, also known as a Variance Ration Criterion. It is defined as the ratio of the between-clusters dispersion mean and the within-cluster dispersion. More information [here](https://scikit-learn.org/stable/modules/clustering.html).

### Comparison to PCA

In the graphs below, we show the difference in spread of data after taking 2 most prominent components of lda and pca. We can observe a much higher explained variance ratio of the first 2 components with lda, as well as a much cleaner spread between classes.

You can play with the parameters but the result will not change much, as we discovered that lda is much more appropriate for processing the data we posses, mostly as the data is labelled. We will talk about the relation between number of users and length of interval segment in the next paragraphs.

In [5]:
# Parameters
number_of_users = 7
segment_interval_length = 45

# Get n random users and segment interval length
users = get_random_users(n=number_of_users)
interval = segment_intervals.index(segment_interval_length)
# Perform and visualize lda
lda, score = get_lda(datas[interval], users, score=True, stats=False)
fig = scatter(lda, x="component_1", y="component_2", color="user", color_continuous_scale='Rainbow', title="LDA TECHNIQUE; number of users: {}, segment interval: {} seconds, score: {}".format(number_of_users, segment_intervals[interval], score))
fig.show()
# Perform and visualize pca
pca, score = get_pca(datas[interval], users, score=True, stats=False)
fig = scatter(pca, x="component_1", y="component_2", color="user", color_continuous_scale='Rainbow', title="PCA TECHNIQUE; number of users: {}, segment interval: {} seconds, score: {}".format(number_of_users, segment_intervals[interval], score))
fig.show()

Users: [8, 10, 24, 19, 23, 7, 15]


### Users and time intervals

The number of users we want to distinguish is closely related to the choice of the time interval segments. Rule of thumb is that with the increasing number of users, we need longer time segments to distinguish between the users. Below is series of graphs for different number of users with different time intervals used, noted with vrc scores.

In [6]:
user_number = 5
users = get_random_users(n=user_number)
scores = []
for data in datas:
    _, score = get_lda(data, users, stats=False, score=True)
    scores.append(score)
fig = make_subplots(rows=2, cols=5, subplot_titles=["{} seconds, {}".format(segment_intervals[i], int(scores[i])) for i in range(0, len(datas))])
for i in range(0, len(datas)):
    lda, score = get_lda(datas[i], users, stats=False, score=True)
    fig.add_trace(go.Scatter(x=lda["component_1"],y=lda["component_2"],mode='markers',marker=dict(color=lda["user"],colorscale='rainbow')),row=int(i/5) + 1,col=i%5 + 1)
fig.update_layout(title="Number of users: {}".format(user_number))
fig.show()

user_number = 9
users = get_random_users(n=user_number)
scores = []
for data in datas:
    _, score = get_lda(data, users, stats=False, score=True)
    scores.append(score)
fig = make_subplots(rows=2, cols=5, subplot_titles=["{} seconds, {}".format(segment_intervals[i], int(scores[i])) for i in range(0, len(datas))])
for i in range(0, len(datas)):
    lda, score = get_lda(datas[i], users, stats=False, score=True)
    fig.add_trace(go.Scatter(x=lda["component_1"],y=lda["component_2"],mode='markers',marker=dict(color=lda["user"],colorscale='rainbow')),row=int(i/5) + 1,col=i%5 + 1)
fig.update_layout(title="Number of users: {}".format(user_number))
fig.show()

user_number = 13
users = get_random_users(n=user_number)
scores = []
for data in datas:
    _, score = get_lda(data, users, stats=False, score=True)
    scores.append(score)
fig = make_subplots(rows=2, cols=5, subplot_titles=["{} seconds, {}".format(segment_intervals[i], int(scores[i])) for i in range(0, len(datas))])
for i in range(0, len(datas)):
    lda, score = get_lda(datas[i], users, stats=False, score=True)
    fig.add_trace(go.Scatter(x=lda["component_1"],y=lda["component_2"],mode='markers',marker=dict(color=lda["user"],colorscale='rainbow')),row=int(i/5) + 1,col=i%5 + 1)
fig.update_layout(title="Number of users: {}".format(user_number))
fig.show()

Users: [22, 10, 27, 9, 25]


Users: [10, 24, 14, 16, 20, 26, 13, 9, 17]


Users: [25, 14, 12, 15, 11, 16, 19, 24, 23, 13, 8, 10, 9]


To better understand how lda behaves with different number of users in relation to different time intervals, we did 500 calculations of lda for different number of users and took the mean vrc score for every time interval. The data is shown on the graph below.

In [8]:
from numpy import array

def calculate_scores(n, epochs=50):
    print("Calculating for {} users.".format(n))
    scores = [[] for _ in range(0, epochs)]
    for i in range(0, epochs):
        users = get_random_users(n=n, stats=False)
        for data in datas:
            _, score = get_lda(data, users, stats=False, score=True)
            scores[i].append(score)
    scores = array(scores).T
    result = []
    for i in range(0, len(datas)):
        result.append([segment_intervals[i], mean(scores[i]), n])
    return DataFrame(result, columns=["time_interval", "score", "users"])

e = 500
df3 = calculate_scores(3, epochs=e)
df5 = calculate_scores(5, epochs=e)
df7 = calculate_scores(7, epochs=e)
df9 = calculate_scores(9, epochs=e)
df11 = calculate_scores(11, epochs=e)
df13 = calculate_scores(13, epochs=e)
df15 = calculate_scores(15, epochs=e)
df17 = calculate_scores(17, epochs=e)

df = concat([df3, df5, df7, df9, df11, df13, df15, df17])
line(df, x="time_interval", y="score", color="users")


Calculating for 3 users.
Calculating for 5 users.
Calculating for 7 users.
Calculating for 9 users.
Calculating for 11 users.
Calculating for 13 users.
Calculating for 15 users.
Calculating for 17 users.


## Helpers

In [4]:
from scipy.signal import find_peaks
from datetime import datetime, timedelta
from math import sqrt
import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import plotly.express as px
from plotly.express import line
from pandas import DataFrame, read_csv, concat
from numpy import mean, std
import plotly.graph_objects as go
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from plotly.express import scatter, line
from sklearn.preprocessing import StandardScaler
import random
from plotly.subplots import make_subplots
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from math import log10

def load_data(seance_id, sens):
    seance = Seance.objects.get(id=seance_id)
    if sens == "accelerometer":
        sensor_ids = [60,61,62]

        sensors = Sensor.objects.filter(id__in=sensor_ids).order_by('id')
        return (
                SensorRecord.objects.filter(seance=seance, sensor=sensors[0]).order_by("timestamp"), 
                SensorRecord.objects.filter(seance=seance, sensor=sensors[1]).order_by("timestamp"), 
                SensorRecord.objects.filter(seance=seance, sensor=sensors[2]).order_by("timestamp")
        )
    elif sens == "gyroscope":
        sensor_ids = [63,64,65]
        sensors = Sensor.objects.filter(id__in=sensor_ids).order_by('id')
        return (
                SensorRecord.objects.filter(seance=seance, sensor=sensors[0]).order_by("timestamp"), 
                SensorRecord.objects.filter(seance=seance, sensor=sensors[1]).order_by("timestamp"), 
                SensorRecord.objects.filter(seance=seance, sensor=sensors[2]).order_by("timestamp")
        )
    elif sens == "force":
        sensor_ids = [54,55,76,77]
        sensors = Sensor.objects.filter(id__in=sensor_ids).order_by('topic')
        return (
                SensorRecord.objects.filter(seance=seance, sensor=sensors[0], value__gte=50).order_by("timestamp"), 
                SensorRecord.objects.filter(seance=seance, sensor=sensors[1], value__gte=50).order_by("timestamp"), 
                SensorRecord.objects.filter(seance=seance, sensor=sensors[2], value__gte=50).order_by("timestamp"), 
                SensorRecord.objects.filter(seance=seance, sensor=sensors[3], value__gte=50).order_by("timestamp")
        )
    elif sens == "cpu":
        sensor_ids = [78,79,80,81]
        sensors = Sensor.objects.filter(id__in=sensor_ids).order_by('topic')
        return (
                SensorRecord.objects.filter(seance=seance, sensor=sensors[0]).order_by("timestamp"), 
                SensorRecord.objects.filter(seance=seance, sensor=sensors[1]).order_by("timestamp"), 
                SensorRecord.objects.filter(seance=seance, sensor=sensors[2]).order_by("timestamp"), 
                SensorRecord.objects.filter(seance=seance, sensor=sensors[3]).order_by("timestamp")
        )
    elif sens == "ram":
        sensor_ids = [82]
        sensors = Sensor.objects.filter(id__in=sensor_ids).order_by('topic')
        return SensorRecord.objects.filter(seance=seance, sensor=sensors[0]).order_by("timestamp")
    elif sens == "net":
        sensor_ids = [83, 84]
        sensors = Sensor.objects.filter(id__in=sensor_ids).order_by('id')
        return (
            SensorRecord.objects.filter(seance=seance, sensor=sensors[0]).order_by("timestamp"),
            SensorRecord.objects.filter(seance=seance, sensor=sensors[1]).order_by("timestamp")
        )
    elif sens == "pir":
        sensor_ids = [58, 59, 66, 67, 68, 69]
        sensors = Sensor.objects.filter(id__in=sensor_ids).order_by('id')
        return (
            SensorRecord.objects.filter(seance=seance, sensor__in=sensors).order_by("timestamp"),
            seance.start,
            seance.end,
        )
    else:
        raise ValueError("Invalid sensor string.")
    
def process_signal(records):
    """
    Take Django query and do basic signal processing.
    """
    values = [x.value for x in records]
    times = [x.timestamp for x in records]
    m = mean(values)
    s = std(values)
    norm = [(x - m)/s for x in values]
    
    return values, times, norm, m, s

def join_accelerometer_signals(x, y, z):
    """
    Join accelerometer signals, based simply on concurrence. 
    We can do this, as only one controller sends data in loop for all axis.
    """
    result = []
    n = min(len(x), len(y), len(z))
    for a,b,c in zip(x[:n], y[:n], z[:n]):
        result.append(sqrt(a**2+b**2+c**2))
    return result, mean(result), std(result)

    
def mean_crossing_rate(signal, m):
    """
    Calculate mean crossing rate from signal.
    Rate of mean crossings vs. the signal length.
    """
    try:
        prev = signal[0]
    except IndexError:
        return 0
    crosses = 0
    length = len(signal) - 1
    
    for curr in signal[1:]:
        if prev <= m < curr or prev > m >=curr:
            crosses += 1
        prev = curr
    if length < 1:
        return 0
    return crosses/length

def mean_acceleration_intensity(signal):
    """
    Mean derivative of a signal.
    """
    try:
        prev = signal[0]
    except IndexError:
        return 0
    length = len(signal) - 1
    derv = []
    
    for curr in signal[1:]:
        derv.append(abs(curr - prev))
        prev = curr
        
    return mean(derv)

def join_cpu_signals(a, b, c, d):
    """
    Similar to accelerometer one.
    """
    result = []
    n = min(len(a), len(b), len(c), len(d))
    for w,x,y,z in zip(a[:n], b[:n], c[:n], d[:n]):
        result.append(sqrt(w**2+x**2+y**2+z**2))
    return result, mean(result), std(result)

def get_cpu_stats(val):
    if not val:
        return 0, 0, 0
    return min(val), max(val), mean_crossing_rate(val, mean(val))
    
    
def find_ram_jump(signal):
    if not signal:
        return [], {}
    derivative = []
    prev = signal[0]
    for curr in signal[1:]:
        derivative.append(abs(curr-prev))
        prev = curr
    peaks, _ = find_peaks(derivative, threshold=0.25)
    
    p = {"position": [], "magnitude": []}
    for x in peaks:
        p["position"].append(x)
        p["magnitude"].append(derivative[x])
    return derivative, p

def get_mem_stats(val, peaks, derivatives):
    # Calculate average inter jump interval
    intervals = []
    if peaks and peaks["position"]:
        prev = peaks["position"][0]
        for curr in peaks["position"][1:]:
            intervals.append(curr-prev)
            prev = curr
    if val:
        avg_load = round(mean(val), 2)
        min_load = min(val)
        max_load = max(val)
    else:
        avg_load = 0
        min_load = 0
        max_load = 0
    if peaks:
        jump_count = len(peaks["position"])
        if derivatives:
            jump_rate = round(len(peaks["position"])/len(derivatives), 2)
        else:
            jump_rate = 0
        avg_jump_value = round(mean(peaks["magnitude"]), 2)
        avg_inter_jump_interval = round(mean(intervals), 2)
    else:
        jump_count = 0
        jump_rate = 0
        avg_jump_value = 0
        avg_inter_jump_interval = 0
    return avg_load, min_load, max_load, jump_count, jump_rate, avg_jump_value, avg_inter_jump_interval

def find_net_jump(signal):
    if not signal:
        return [], {}
    derivative = []
    prev = signal[0]
    for curr in signal[1:]:
        derivative.append(abs(curr-prev))
        prev = curr
    peaks, _ = find_peaks(derivative, threshold=mean(derivative))

    p = {"position": [], "magnitude": []}
    for x in peaks:
        p["position"].append(x)
        p["magnitude"].append(derivative[x])
    return derivative, p

def get_net_stats(val, peaks, derivatives):
    # Calculate average inter jump interval
    intervals = []
    if peaks and peaks["position"]:
        prev = peaks["position"][0]
        for curr in peaks["position"][1:]:
            intervals.append(curr-prev)
            prev = curr
    try:
        sum_load = val[-1] - val[0]
    except IndexError:
        sum_load = 0
    if peaks:
        jump_count = len(peaks["position"])
        if derivatives:
            jump_rate = round(len(peaks["position"])/len(derivatives), 2)
        else:
            jump_rate = 0
        avg_jump_value = round(mean(peaks["magnitude"]), 2)
        avg_inter_jump_interval = round(mean(intervals), 2)
    else:
        jump_count = 0
        jump_rate = 0
        avg_jump_value = 0
        avg_inter_jump_interval = 0
    return sum_load, jump_count, jump_rate, avg_jump_value, avg_inter_jump_interval

def get_random_users(n=3, stats=True):
    users = []
    while len(users) < n:
        user = random.randint(7,27)
        if user not in users:
            users.append(user)
    if stats:
        print("Users: {}".format(users))
    return users
    
def get_lda(data, users, comp_num=2, stats=True, score=False):
    data = data[data.user.isin(users)]
    X = data.iloc[:, 2:].values
    y = data.iloc[:, 0].values.ravel()
    z = data.iloc[:, 1].values.ravel()
    
    lda = LinearDiscriminantAnalysis(n_components=comp_num)
    X_lda = lda.fit_transform(X, y)
    score = round(metrics.calinski_harabasz_score(X_lda, y))
    if stats:
        print(lda.explained_variance_ratio_)
        print(score)
            
    y = y.reshape(len(y), 1)
    z = z.reshape(len(z), 1)
    df = DataFrame([list(y) + list(z) + list(x) for x, y, z in zip(X_lda, y, z)], columns=["user", "try", "component_1", "component_2"])
    if score:
        return df, score
    return df

def get_pca(data, users, comp_num=2, stats=True, score=False):
    data = data[data.user.isin(users)]
    X = data.iloc[:, 2:].values
    y = data.iloc[:, 0].values.ravel()
    z = data.iloc[:, 1].values.ravel()
    
    scaler = StandardScaler()
    scaler.fit(X)
    X=scaler.transform(X)
    pca = PCA(n_components=comp_num)
    X_pca = pca.fit_transform(X)
    score = round(metrics.calinski_harabasz_score(X_pca, y))
    if stats:
        print(pca.explained_variance_ratio_)
        print(score)
    
    y = y.reshape(len(y), 1)
    z = z.reshape(len(z), 1)
    df = DataFrame([list(y) + list(z) + list(x) for x, y, z in zip(X_pca, y, z)], columns=["user", "try", "component_1", "component_2"])
    if score:
        return df, score
    return df